In [101]:
import pandas as pd
import os
import subprocess

In [102]:
hosts = pd.read_csv("../common/hosts.csv")
supressed_columns = ['ip','user']

select_hosts = hosts
select_hosts.drop(columns=supressed_columns)
select_hosts['ip'].to_csv(r'ip.txt', header=False, index=None, sep=' ')

In [103]:
columns_memory = ['hostname','memory[MB]']
memory_df = pd.DataFrame(columns = columns_memory)

for index, row in hosts.iterrows():
    result = subprocess.check_output('ssh -n -o StrictHostKeyChecking=no ' +row['user'] +'@'+row['ip']  +' free | grep Mem || :', shell=True)
    if(len(result) > 1):
        memory_row = result.split()
        memory_df.loc[len(memory_df)] = [ row['hostname'],round(int(memory_row[1])/1024)]

        
memory_df.to_csv(r'memory.txt', header=False, index=None, sep=',')
memory_df

,hostname,memory[MB]
0,pi-node2,926
1,pi-node3,926
2,kx-Lenovo-H520g,15864
3,nuc-01,32037
4,nuc-02,32037
5,nuc-03,32037
6,pi-node4,3906
7,pi-node5,3906
8,pi-node7,3906
9,pi-node8,3906


In [104]:
columns_cores = ['hostname','cores']
cores_df = pd.DataFrame(columns = columns_cores)

for index, row in hosts.iterrows():
    result = subprocess.check_output('ssh -n -o StrictHostKeyChecking=no ' +row['user'] +'@'+row['ip']  +' getconf _NPROCESSORS_ONLN || :', shell=True)
    if(len(result) > 1):
        core_row = result
        cores_df.loc[len(cores_df)] = [ row['hostname'],int(result)]

        
cores_df.to_csv(r'cores.txt', header=False, index=None, sep=',')
cores_df

,hostname,cores
0,pi-node2,4
1,pi-node3,4
2,kx-Lenovo-H520g,8
3,nuc-01,8
4,nuc-02,8
5,nuc-03,8
6,pi-node4,4
7,pi-node5,4
8,pi-node7,4
9,pi-node8,4


In [105]:
columns_disk = ['hostname','diskSize[MB]','usage', 'mountpoint']
disk_df = pd.DataFrame(columns = columns_disk)

for index, row in hosts.iterrows():
    result = subprocess.check_output('ssh -n -o StrictHostKeyChecking=no ' +row['user'] +'@'+row['ip']  +' df  | grep ^/dev || :', shell=True, encoding='utf-8')
    if(len(result) > 1):
        for result_row in result.splitlines():
            result_row_split = result_row.split()
            disk_df.loc[len(disk_df)] = [ row['hostname'],round(int(int(result_row_split[1])/1024/1024)),result_row_split[4],result_row_split[5]]

        
disk_df.to_csv(r'disk.txt', header=False, index=None, sep=',')
disk_df

,hostname,diskSize[MB],usage,mountpoint
0,pi-node2,28,6%,/
1,pi-node2,0,16%,/boot
2,pi-node3,117,2%,/
3,pi-node3,0,16%,/boot
4,kx-Lenovo-H520g,916,11%,/
5,nuc-01,915,3%,/
6,nuc-01,0,1%,/boot/efi
7,nuc-01,4620,1%,/media/kaxios/hdddisk
8,nuc-02,915,3%,/
9,nuc-02,0,1%,/boot/efi


In [106]:
merged_detail = pd.merge(cores_df, memory_df, on='hostname', how='inner')
merged_detail

,hostname,cores,memory[MB]
0,pi-node2,4,926
1,pi-node3,4,926
2,kx-Lenovo-H520g,8,15864
3,nuc-01,8,32037
4,nuc-02,8,32037
5,nuc-03,8,32037
6,pi-node4,4,3906
7,pi-node5,4,3906
8,pi-node7,4,3906
9,pi-node8,4,3906


In [107]:
disk_df['diskSize[MB]'].sum(axis = 0, skipna = True) 

14947